# MAPC Team 1 Notebook

In [1]:
# code block for imports 
import pandas as pd 
import numpy as np
import sqlite3
from sqlalchemy import create_engine

In [2]:
# functions for reading data as a df and inserting into sql db
def readData(filename):
    data = pd.read_csv(filename)
    return data

def insertDataToDB(tablename, df,conn):
    df.to_sql(tablename,conn, if_exists="replace")


In [3]:
# connect to database cs506MAPC.db
conn = sqlite3.connect('cs506MAPC.db')

# read data from FCC June 2019 data. 
# If this line causes errors for you, you first need to download the data and 
# and unzip it into your local copy of your repository. (Can't upload the CSV to git because it's too large)
# to download the zip file, go here --> https://github.com/MAPC/broadband-data-bu/tree/main/FCC%20data
# NOTE: Make sure that you have the .csv file in the 'MAPC Broadband Equity - Team 1' folder and that you rename it
# to "fcc_data_june2019.csv" 

df = readData("fcc_data_june2019.csv")
insertDataToDB("fcc_data_t",df,conn)

## Display first 10 Providers with no service (0 upload and 0 download)

In [4]:
qrystr ="SELECT DISTINCT ProviderName,substr(BlockCode,0,12) as tractNum,MaxAdDown,MaxAdUp FROM FCC_DATA_T ORDER BY substr(BlockCode,0,12) ASC"


#convert query results into dataframe
df_filtered_providers = pd.read_sql_query(qrystr, conn)
print(df_filtered.head(10))




                        ProviderName     tractNum  MaxAdDown  MaxAdUp
0     MCI Communications Corporation  25001010100        0.0    0.000
1                       ViaSat, Inc.  25001010100       35.0    3.000
2             U.S. TelePacific Corp.  25001010100        0.0    0.000
3  COMCAST CABLE COMMUNICATIONS, LLC  25001010100      987.0   35.000
4  COMCAST CABLE COMMUNICATIONS, LLC  25001010100        0.0    0.000
5            EarthLink Business, LLC  25001010100        0.0    0.000
6        PAETEC Communications, Inc.  25001010100        0.0    0.000
7           Verizon New England Inc.  25001010100       10.0    1.000
8           Verizon New England Inc.  25001010100       15.0    1.000
9           Verizon New England Inc.  25001010100        7.0    0.768


In [12]:
qryStr2 = "SELECT DISTINCT ProviderName, substr(BlockCode,0,12) as tractNum, MaxAdDown, MaxAdUp FROM FCC_DATA_T WHERE MaxAdDown != 0.0 AND MaxAdUp != 0.0 ORDER BY substr(BlockCode,0,12) ASC"
df_filtered_non_zero_providers = pd.read_sql_query(qryStr2, conn)
print(df_filtered_non_zero_providers.head(10))


                        ProviderName     tractNum  MaxAdDown  MaxAdUp
0                       ViaSat, Inc.  25001010100       35.0    3.000
1  COMCAST CABLE COMMUNICATIONS, LLC  25001010100      987.0   35.000
2           Verizon New England Inc.  25001010100       10.0    1.000
3           Verizon New England Inc.  25001010100       15.0    1.000
4           Verizon New England Inc.  25001010100        7.0    0.768
5           Verizon New England Inc.  25001010100        5.0    0.768
6           Verizon New England Inc.  25001010100        1.5    0.384
7           Verizon New England Inc.  25001010100        3.0    0.768
8               HNS License Sub, LLC  25001010100       25.0    3.000
9                  VSAT Systems, LLC  25001010100        2.0    1.300


In [5]:
df = readData("median_income.csv")
print(df.head(10))
df_filtered = df[['ct10_id','mhi','mhi_me','o_mhi','o_mhi_me','r_mhi','r_mhi_me']]
insertDataToDB("median_income_t",df_filtered,conn)


#todo: filter data from fcc data which has 0 max_ad_up and max_ad_down
#this means company doesnt supply that area. 

   seq_id      ct10_id               geoid  logrecno acs_year    mhi  mhi_me  \
0   11825  25001010100  14000US25001010100       911  2014-18  50741    7720   
1   11826  25001010206  14000US25001010206       912  2014-18  69267    7140   
2   11827  25001010208  14000US25001010208       913  2014-18  65446   10571   
3   11828  25001010304  14000US25001010304       914  2014-18  79044   14744   
4   11829  25001010306  14000US25001010306       915  2014-18  62553   10246   
5   11830  25001010400  14000US25001010400       916  2014-18  60188   15938   
6   11831  25001010500  14000US25001010500       917  2014-18  78682   18270   
7   11832  25001010600  14000US25001010600       918  2014-18  73203   12880   
8   11833  25001010700  14000US25001010700       919  2014-18  82417   20490   
9   11834  25001010800  14000US25001010800       920  2014-18  71776    8325   

   o_mhi  o_mhi_me  r_mhi  r_mhi_me  
0  57109      9886  32930      8570  
1  71019      4989  31023     20026  
2  66